# Consultas centradas no termo

Quando os campos vêm diretamente do modelo de dados de origem, os cálculos TF × IDF que resultam de pesquisas não se alinham perfeitamente às expectativas do usuário. TF × IDF pode ser uma métrica ruim quando o campo e a proporção aproximada subjacente dos recursos  não são mapeados para signals que os usuários se preocupam ao classificar. Quando você pesquisa em dezenas de campos, simplesmente porque "é isso que está no banco de dados", você acaba com muitas dessas anomalias de pontuação que não correspondem às expectativas do usuário.

A pesquisa centrada em campo amplifica a discordância de signal. Ao calcular o score de cada campo isoladamente, a pesquisa centrada em campo é propensa a influenciar fortemente os resultados da pesquisa em uma direção ou outra. Ao executar pesquisas centradas em campo em dezenas e dezenas de campos, todos os critérios possíveis do seu banco de dados, você procura por problemas.

O modelo de dados de origem mantém você preso em uma visualização bottom-up, modelo de dados de origem primeiro.
Para fazer uma boa modelagem de signals, é necessário pensar de top-down e primeiramente no usuário. Com o que os usuários se preocupam ao classificar? Como você pode criar campos do modelo de dados de origem para calcular esses signals? Qual deve ser a frequência do documento de william shatner para refletir o senso de conscientização do termo pelo usuário?

Como você verá, a pesquisa centrada em termos pode ajudar a fornecer essa perspectiva de cima para baixo na modelagem de signals.

# Inicialização

In [1]:
#PARÂMETROS

#Máquina e porta (formato host:port)
SOLR_ADDR='localhost:8983'

In [3]:
import json
import requests
import pandas
from datetime import datetime
headers = {'content-type': 'application/json;charset=UTF-8'}

def date_diff_in_seconds(dt2, dt1):
    timedelta = dt2 - dt1
    return timedelta.days * 24 * 3600 + timedelta.seconds

# Some utilities for flattening the explain into something a bit more
# readable. Pass Explain JSON, get something readable (ironically this is what Solr's default output is :-p)
def flatten(l):
    [item for sublist in l for item in sublist]

def simplerExplain(explainJson, depth=0):
    result = " " * (depth * 2) + "%s, %s\n" % (explainJson['value'], explainJson['description'])
    #print json.dumps(explainJson, indent=True)
    if 'details' in explainJson:
        for detail in explainJson['details']:
            result += simplerExplain(detail, depth=depth+1)
    return result

In [4]:
def get_cast_list(cast, n):
    i = 0
    cast_list = []
    for e in cast:
        i += 1
        cast_list.append(e['name'])
        #if i > n:
            #cast_list.append('...')
            #break
    return cast_list

In [ ]:
#Cria um índice novo no Solr e reindexa os dados
def reindex_solr(movieDict={}, delete=True):
    if delete:
        resp = requests.get("http://" + SOLR_ADDR + "/solr/admin/collections?action=DELETE&name=tmdb")
        resp = requests.get("http://" + SOLR_ADDR + "/solr/admin/collections?action=CREATE&name=tmdb&numShards=1")
        print("solr building...", resp.status_code)
    
    movies = ""
    
    for id, movie in movieDict.items():
        movies += json.dumps(movie) + ","
    
    bulkMovies = "[" + movies + "]"

    print("solr indexing...")
    resp = requests.post("http://" + SOLR_ADDR + "/solr/tmdb/update/json/docs?commit=true", data=bulkMovies, headers=headers)
    print("solr indexing done.", resp.status_code)

In [ ]:
#Faz a pesquisa especificada no Solr e imprime os resultados 
def search_solr(usersSearch, qf='title^10 overview'):
    url = 'http://' + SOLR_ADDR + '/solr/tmdb/select?q='+ usersSearch + '&defType=edismax&qf=' + qf + '&rows=30&wt=json&fl=title,score'
    httpResp = requests.get(url, headers=headers) #A
    searchHits = json.loads(httpResp.text)['response']['docs']
    print("Solr results")
    print("Num\tRelevance Score\t\tMovie Title") #B
    for idx, hit in enumerate(searchHits):
        print ("%s\t%s\t\t%s" % (idx + 1, hit['score'], hit['title']))
    print("\n")

In [10]:
#Faz a pesquisa especificada no Elasticsearch e imprime os resultados
def search_elastic(usersSearch, query=None):
    if not query:
        query = {
            'query': {
                'multi_match': { 
                    'query': usersSearch, #A
                    'fields': ['title^10', 'overview'] #B
                }
            },
            'size': '30'
        }
    
    url = 'http://'+ ELASTIC_ADDR +'/tmdb/_search'
    httpResp = requests.get(url, data=json.dumps(query), headers=headers) #A
    searchHits = json.loads(httpResp.text)['hits']
    print("Elasticsearch results")
    lista_resultados = []
    for idx, hit in enumerate(searchHits['hits']):
        filme = [idx + 1, hit['_score'], hit['_source']['title'], hit['_source']['overview'], get_cast_list(hit['_source']['cast'],10), get_cast_list(hit['_source']['directors'],5)]
        lista_resultados.append(filme)
    
    pd.set_option('display.max_colwidth', -1)
    
    df = pd.DataFrame(lista_resultados,columns=['Num', 'Relevance Score', 'Movie Title', 'Overview', 'Cast', 'Director'], index=None)
    return df

In [52]:
usersSearch = 'jornada nas estrelas patrick stewart'
query = {
    'query': {
        'multi_match': { 
            'query': usersSearch,  #User's query
            'fields': ['title', 'overview', 'cast.name.bigramed', 'directors.name.bigramed'],      
            'type': 'cross_fields'
         }
    },
    'size': 50,
    'explain': True
}

df = search_elastic(usersSearch, query)
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Elasticsearch results


,Relevance Score,Movie Title
0,11.073451,X-Men: O Confronto Final
1,10.081892,Acima das Nuvens
2,9.486658,Logan
3,9.379991,Stardust - O Mistério da Estrela
4,8.673704,A Estrela de Belém
5,8.444677,A Espaçonave Das Loucas
6,7.940823,Um Monstro em Paris
7,7.925139,X-Men: Dias de um Futuro Esquecido
8,7.879263,Ad Astra - Rumo às Estrelas
9,7.545320,Jornada nas Estrelas: Generations


In [53]:
query['explain'] = True
httpResp = requests.get('http://'+ ELASTIC_ADDR +'/tmdb/_search', data=json.dumps(query), headers=headers)
jsonResp = json.loads(httpResp.text)
print("Explain for %s" % jsonResp['hits']['hits'][0]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][0]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][1]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][1]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][2]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][2]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][3]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][3]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][25]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][25]['_explanation']))

Explain for X-Men: O Confronto Final
11.073451, max of:
  3.8974152, sum of:
    3.8974152, max of:
      3.8974152, weight(cast.name.bigramed:patrick stewart in 4265) [PerFieldSimilarity], result of:
        3.8974152, score(freq=1.0), computed as boost * idf * tf from:
          2.2, boost
          5.4240685, idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:
            32, n, number of documents containing term
            7370, N, total number of documents with field
          0.3266095, tf, computed as freq / (freq + k1 * (1 - b + b * dl / avgdl)) from:
            1.0, freq, occurrences of term within document
            1.2, k1, term saturation parameter
            0.75, b, length normalization parameter
            64.0, dl, length of field (approximate)
            32.694572, avgdl, average length of field
  11.073451, sum of:
    5.235121, max of:
      5.235121, weight(overview:patrick in 4265) [PerFieldSimilarity], result of:
        5.235121, score(freq=1.0), co

In [25]:
def explain_elastic(users_search, query):
    httpResp = requests.get('http://'+ ELASTIC_ADDR +'/tmdb/_validate/query?explain',data=json.dumps(query), headers=headers)
    print('Explicação da query no Elasticsearch:')
    json_str= json.dumps(json.loads(httpResp.text), indent=2, ensure_ascii=False).encode('utf-8')
    print(json_str.decode())
    print('\n')

In [56]:
query = {
    'query': {
        'multi_match': { 
            'query': usersSearch,  #User's query
            'fields': ['title', 'overview', 'cast.name.bigramed', 'directors.name.bigramed'],      
            'type': 'cross_fields'
         }
    },
}

explain_elastic(usersSearch,query)

Explicação da query no Elasticsearch:
{
  "_shards": {
    "total": 1,
    "successful": 1,
    "failed": 0
  },
  "valid": true,
  "explanations": [
    {
      "index": "tmdb",
      "valid": true,
      "explanation": "((blended(terms:[directors.name.bigramed:jornad _, cast.name.bigramed:jornad _]) blended(terms:[directors.name.bigramed:_ estrel, cast.name.bigramed:_ estrel]) blended(terms:[directors.name.bigramed:estrel patrick, cast.name.bigramed:estrel patrick]) blended(terms:[directors.name.bigramed:patrick stewart, cast.name.bigramed:patrick stewart])) | (blended(terms:[overview:jornad, title:jornad]) blended(terms:[overview:estrel, title:estrel]) blended(terms:[overview:patrick, title:patrick]) blended(terms:[overview:stewart, title:stewart])))"
    }
  ]
}


